In [1]:
# !wget https://raw.githubusercontent.com/attardi/wikiextractor/master/WikiExtractor.py
# !wget http://dumps.wikimedia.org/mswiki/latest/mswiki-latest-pages-articles.xml.bz2
# !bzip2 -d mswiki-latest-pages-articles.xml.bz2
# !python3 WikiExtractor.py --json mswiki-latest-pages-articles.xml

In [2]:
from glob import glob

files = glob('text/*/*')
len(files)

270

In [3]:
import json

with open(files[0]) as fopen:
    x = fopen.readlines()

In [4]:
import re
from unidecode import unidecode

alphabets = '([A-Za-z])'
prefixes = (
    '(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt|Puan|puan|Tuan|tuan|sir|Sir)[.]'
)
suffixes = '(Inc|Ltd|Jr|Sr|Co|Mo)'
starters = '(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever|Dia|Mereka|Tetapi|Kita|Itu|Ini|Dan|Kami|Beliau|Seri|Datuk|Dato|Datin|Tuan|Puan)'
acronyms = '([A-Z][.][A-Z][.](?:[A-Z][.])?)'
websites = '[.](com|net|org|io|gov|me|edu|my)'
another_websites = '(www|http|https)[.]'
digits = '([0-9])'
before_digits = '([Nn]o|[Nn]ombor|[Nn]umber|[Kk]e|=|al)'
month = '([Jj]an(?:uari)?|[Ff]eb(?:ruari)?|[Mm]a(?:c)?|[Aa]pr(?:il)?|Mei|[Jj]u(?:n)?|[Jj]ula(?:i)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]kt(?:ober)?|[Nn]ov(?:ember)?|[Dd]is(?:ember)?)'


def split_into_sentences(text, minimum_length = 5):
    text = text.replace('\x97', '\n')
    text = '. '.join([s for s in text.split('\n') if len(s)])
    text = text + '.'
    text = unidecode(text)
    text = ' ' + text + '  '
    text = text.replace('\n', ' ')
    text = re.sub(prefixes, '\\1<prd>', text)
    text = re.sub(websites, '<prd>\\1', text)
    text = re.sub(another_websites, '\\1<prd>', text)
    text = re.sub('[,][.]+', '<prd>', text)
    if '...' in text:
        text = text.replace('...', '<prd><prd><prd>')
    if 'Ph.D' in text:
        text = text.replace('Ph.D.', 'Ph<prd>D<prd>')
    text = re.sub('[.]\s*[,]', '<prd>,', text)
    text = re.sub(before_digits + '\s*[.]\s*' + digits, '\\1<prd>\\2', text)
    text = re.sub(month + '[.]\s*' + digits, '\\1<prd>\\2', text)
    text = re.sub('\s' + alphabets + '[.][ ]+', ' \\1<prd> ', text)
    text = re.sub(acronyms + ' ' + starters, '\\1<stop> \\2', text)
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]' + alphabets + '[.]',
        '\\1<prd>\\2<prd>\\3<prd>',
        text,
    )
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]', '\\1<prd>\\2<prd>', text
    )
    text = re.sub(' ' + suffixes + '[.][ ]+' + starters, ' \\1<stop> \\2', text)
    text = re.sub(' ' + suffixes + '[.]', ' \\1<prd>', text)
    text = re.sub(' ' + alphabets + '[.]', ' \\1<prd>', text)
    text = re.sub(digits + '[.]' + digits, '\\1<prd>\\2', text)
    if '”' in text:
        text = text.replace('.”', '”.')
    if '"' in text:
        text = text.replace('."', '".')
    if '!' in text:
        text = text.replace('!"', '"!')
    if '?' in text:
        text = text.replace('?"', '"?')
    text = text.replace('.', '.<stop>')
    text = text.replace('?', '?<stop>')
    text = text.replace('!', '!<stop>')
    text = text.replace('<prd>', '.')
    sentences = text.split('<stop>')
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences if len(s) > minimum_length]
    return sentences

split_into_sentences('Pembolehubah yang ketiga adalah niat yang merujuk kepada niat seseorang dalam melakukan pelbagai tingkah laku ( Fishbein et al . 1975 : 12 ), ')

['Pembolehubah yang ketiga adalah niat yang merujuk kepada niat seseorang dalam melakukan pelbagai tingkah laku ( Fishbein et al.1975 : 12 ), .']

In [10]:
from tqdm import tqdm

texts = []

for i in tqdm(range(len(files))):
    with open(files[i]) as fopen:
        s = fopen.readlines()
    
    for i in s:
        i = json.loads(i)
        texts.extend(split_into_sentences(i['text'].replace('<br>', ' ')))
        texts.append('')

100%|██████████| 270/270 [02:33<00:00,  1.76it/s]


In [11]:
with open('dumping-wiki.txt', 'w') as fopen:
    fopen.write('\n'.join(texts))

In [15]:
!head -n 1000 dumping-wiki.txt

Klaas-Jan Huntelaar.
Dirk Jan Klaas "Klaas-Jan" Huntelaar (lahir 12 Ogos 1983) merupakan pemain bola sepak Belanda yang bermain di posisi penyerang.
Beliau kini bermain untuk kelab Ajax.

Hypo-Arena.
Hypo-Arena (dahulu dikenali sebagai ) ialah sebuah stadium serba guna di Klagenfurt, Austria.
Ia merupakan stadium pasukan SK Austria Karnten.
Stadium lama dikenali sebagai Wortherseestadion, dibina pada 1960 dan mempunyai kapasiti sebanyak 10,900.
Ia dirobohkan pada 2005 dan digantikan dengan Hypo-Arena yang baru, juga dikenali sehingga 30 Jun 2007 dengan nama "Wortherseestadion".
Ia adalah salah satu daripada 8 stadium untuk UEFA Euro 2008, dan dibina untuk menampung 32,000 penonton.
Selepas acara tersebut, kapasiti stadium ini sedang dipertimbangkan untuk dikurangkan kepada 12,500.
Stadium ini dibuka secara rasmi pada 7 September 2007 dengan menjadi tuan rumah untuk perlawanan persahabatan di antara Austria dan Jepun di hadapan 26,500 penonton.

Marta Vieira da Silva.
Marta Vieira da Si